In [1]:
import n2d2
from n2d2.cells.nn import Fc, Conv, Pool2d
import math
import datetime
import numpy as np
import pickle
import os

## N-MNIST (Accumulation 300 events)

| Representation | Accuracy |
| ------------------- | -------- |
| 2 Channels             | 95.29%      | 
| 3 Channels             | 94.50%      | 
| positive + negative              | 93.98%      |
| positive - negative            | 94.85%      | 
| only positive            | 93.21%      | 
| only negative            | 93.78%      |

Note: Initially, we kept the input dimensions at 28x28. However, after opting for the 2-channel representation, we adjusted the input size to 32x32. In the latter scenario, as detailed in the other Jupyter notebook file, we achieved an accuracy of 95.43% instead of 95.29%.     

In [2]:
solver_conf = n2d2.ConfigSection(
    learning_rate=0.001,
)

def conv_conf():
    return n2d2.ConfigSection(
        activation=n2d2.activation.Rectifier(),
        no_bias=True,
        weights_solver=n2d2.solver.Adam(**solver_conf),
        bias_solver=n2d2.solver.Adam(**solver_conf),
    )
    
def fc_conf1():
    return n2d2.ConfigSection(
        activation=n2d2.activation.Rectifier(),
        no_bias=True,
        weights_solver=n2d2.solver.Adam(**solver_conf),
        bias_solver=n2d2.solver.Adam(**solver_conf),
    )
def fc_conf2():
    return n2d2.ConfigSection(
        activation=n2d2.activation.Linear(),
        no_bias=True,
        weights_solver=n2d2.solver.Adam(**solver_conf),
        bias_solver=n2d2.solver.Adam(**solver_conf),
    )
    
# Definition of layers for quantization

def conv_quantization_conf(n_bits):
    return n2d2.ConfigSection(
        activation=n2d2.activation.Rectifier(),
        no_bias=True,
        weights_solver=n2d2.solver.Adam(**solver_conf),
        bias_solver=n2d2.solver.Adam(**solver_conf),
        quantizer=n2d2.quantizer.SATCell(
            apply_scaling=True,
            apply_quantization=True,
            range=2**n_bits-1,
        ),
    )

def fc_quantization_conf1(n_bits):
    return n2d2.ConfigSection(
        activation=n2d2.activation.Rectifier(),
        no_bias=True,
        weights_solver=n2d2.solver.Adam(**solver_conf),
        bias_solver=n2d2.solver.Adam(**solver_conf),
        quantizer=n2d2.quantizer.SATCell(
            apply_scaling=True,
            apply_quantization=True,
            range=2**n_bits-1,
        ),
    )
def fc_quantization_conf2(n_bits):
    return n2d2.ConfigSection(
        activation=n2d2.activation.Linear(),
        no_bias=True,
        weights_solver=n2d2.solver.Adam(**solver_conf),
        bias_solver=n2d2.solver.Adam(**solver_conf),
        quantizer=n2d2.quantizer.SATCell(
            apply_scaling=True,
            apply_quantization=True,
            range=2**n_bits-1,
        ),
    )

## N-MNIST : 2 channels (positive events, negative events)

In [3]:
with open('data_final/data_2_channels.pkl', 'rb') as file:
    data_2_channels = pickle.load(file)
with open('data_final/labels_2_channels.pkl', 'rb') as file:
    labels_2_channels = pickle.load(file)

In [ ]:
db = n2d2.database.Numpy(random_partitioning=False)

db.load(data_2_channels, labels_2_channels)
db.partition_stimuli(5/7, 1/7, 1/7) # training: 50k, validation: 10k, test: 10k

print("\n### Create Provider ###")
batch_size = 64
provider = n2d2.provider.DataProvider(db, [28, 28, 2], batch_size=batch_size)
print(provider)
target = n2d2.target.Score(provider)

In [ ]:
print("\n### Loading Model (without quantization) ###")
model_2_channels = n2d2.cells.Sequence([
    Conv(2, 6, kernel_dims=[5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(6, 16, [5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(16, 120, [4, 4], **conv_conf()),
    Fc(120, 84, **fc_conf1()),
    Fc(84, 10, **fc_conf2()),
])
print(model_2_channels)
softmax = n2d2.cells.Softmax(with_loss=True)

In [ ]:
nb_epochs = 15
print("\n### Training ###")

start_training_time = datetime.datetime.now()
print("Start time Training: " + str(start_training_time))
for epoch in range(nb_epochs):
    provider.set_partition("Learn")
    model_2_channels.learn()
    print("\n# Train Epoch: " + str(epoch) + " #")
    for i in range(math.ceil(db.get_nb_stimuli('Learn')/batch_size)):
        x = provider.read_random_batch()
        x = model_2_channels(x)
        x = softmax(x)
        x = target(x)
        x.back_propagate()
        x.update()
        print("Example: " + str(i * batch_size) + ", loss: "+ "{0:.3f}".format(x[0]), end='\r')
        
    print("\n### Validation ###")
    target.clear_success()
    provider.set_partition('Validation')
    model_2_channels.test()
    for i in range(math.ceil(db.get_nb_stimuli('Validation') / batch_size)):
        batch_idx = i*batch_size
        x = provider.read_batch(batch_idx)
        x = model_2_channels(x)
        x = softmax(x)
        x = target(x)
        print("Test: " + str(i * batch_size) + ", success: "+ "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')
        
print("\n")        
end_training_time = datetime.datetime.now()
print("End time Training: " + str(end_training_time))
training_time = end_training_time - start_training_time
minutes, seconds = divmod(training_time.total_seconds(), 60)
print("Training time: " + str(int(minutes)) + " min " + str(int(seconds)) + " sec ")

In [6]:
model_2_channels.import_free_parameters("./model_2_channels_old", ignore_not_exists=True)
target = n2d2.target.Score(provider)
provider.set_partition('Test')
print("\n### Testing ###")

model_2_channels.test()

num_tests = 1
total_inference_time = datetime.timedelta()

for test_iteration in range(num_tests):
    print(f"\n### Testing - Iteration {test_iteration + 1}/{num_tests} ###\n")
    start_testing_time = datetime.datetime.now()
    for i in range(math.ceil(provider.get_database().get_nb_stimuli('Test')/batch_size)):
        batch_idx = i*batch_size
    
        x = provider.read_batch(batch_idx)
        x = model_2_channels(x)
        x = softmax(x)
        x = target(x)
    
        print("Example: " + str(i * batch_size) + ", test success: "
              + "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')
    
    end_testing_time = datetime.datetime.now()
    inference_time = end_testing_time - start_testing_time
    total_inference_time += inference_time
    minutes, seconds = divmod(inference_time.total_seconds(), 60)
    print("\nInference time: " + str(int(minutes)) + " min " + str(seconds) + " sec")

average_inference_time = total_inference_time / num_tests
avg_minutes, avg_seconds = divmod(average_inference_time.total_seconds(), 60)
print("\nAverage Inference time over {} iterations: {} min {} sec".format(num_tests, int(avg_minutes), avg_seconds))

# save a confusion matrix
#target.log_confusion_matrix("model_2_channels_old")
# Exporting weights #
#x.get_deepnet().export_network_free_parameters("./model_2_channels_old")

Import ./model_2_channels_old/Conv_0.syntxt
Import ./model_2_channels_old/Conv_1.syntxt
Import ./model_2_channels_old/Conv_2.syntxt
Import ./model_2_channels_old/Fc_0.syntxt
Import ./model_2_channels_old/Fc_1.syntxt

### Testing ###

### Testing - Iteration 1/1 ###

Example: 9984, test success: 95.29%
Inference time: 0 min 4.672638 sec

Average Inference time over 1 iterations: 0 min 4.672638 sec


## N-MNIST : 3 channels (positive events, negative events, average timestamp)

In [7]:
with open('data_final/data_3_channels.pkl', 'rb') as file:
    data_3_channels = pickle.load(file)
with open('data_final/labels_3_channels.pkl', 'rb') as file:
    labels_3_channels = pickle.load(file)

In [ ]:
db = n2d2.database.Numpy(random_partitioning=False)
db.load(data_3_channels, labels_3_channels)
db.partition_stimuli(5/7, 1/7, 1/7) # training: 50k, validation: 10k, test: 10k

print("\n### Create Provider ###")
batch_size = 64
provider = n2d2.provider.DataProvider(db, [28, 28, 3], batch_size=batch_size)
print(provider)
target = n2d2.target.Score(provider)

In [ ]:
print("\n### Loading Model (without quantization) ###")
model_3_channels = n2d2.cells.Sequence([
    Conv(3, 6, kernel_dims=[5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(6, 16, [5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(16, 120, [4, 4], **conv_conf()),
    Fc(120, 84, **fc_conf1()),
    Fc(84, 10, **fc_conf2()),
])
print(model_3_channels)
softmax = n2d2.cells.Softmax(with_loss=True)

In [ ]:
nb_epochs = 15
print("\n### Training ###")

start_training_time = datetime.datetime.now()
print("Start time Training: " + str(start_training_time))
for epoch in range(nb_epochs):
    provider.set_partition("Learn")
    model_3_channels.learn()
    print("\n# Train Epoch: " + str(epoch) + " #")
    for i in range(math.ceil(db.get_nb_stimuli('Learn')/batch_size)):
        x = provider.read_random_batch()
        x = model_3_channels(x)
        x = softmax(x)
        x = target(x)
        x.back_propagate()
        x.update()
        print("Example: " + str(i * batch_size) + ", loss: "+ "{0:.3f}".format(x[0]), end='\r')
        
    print("\n### Validation ###")
    target.clear_success()
    provider.set_partition('Validation')
    model_3_channels.test()
    for i in range(math.ceil(db.get_nb_stimuli('Validation') / batch_size)):
        batch_idx = i*batch_size
        x = provider.read_batch(batch_idx)
        x = model_3_channels(x)
        x = softmax(x)
        x = target(x)
        print("Test: " + str(i * batch_size) + ", success: "+ "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')
        
print("\n")        
end_training_time = datetime.datetime.now()
print("End time Training: " + str(end_training_time))
training_time = end_training_time - start_training_time
minutes, seconds = divmod(training_time.total_seconds(), 60)
print("Training time: " + str(int(minutes)) + " min " + str(int(seconds)) + " sec ")

In [10]:
model_3_channels.import_free_parameters("./model_3_channels", ignore_not_exists=True)
target = n2d2.target.Score(provider)
provider.set_partition('Test')
print("\n### Testing ###")

model_3_channels.test()

num_tests = 1
total_inference_time = datetime.timedelta()

for test_iteration in range(num_tests):
    print(f"\n### Testing - Iteration {test_iteration + 1}/{num_tests} ###\n")
    start_testing_time = datetime.datetime.now()
    for i in range(math.ceil(provider.get_database().get_nb_stimuli('Test')/batch_size)):
        batch_idx = i*batch_size
    
        x = provider.read_batch(batch_idx)
        x = model_3_channels(x)
        x = softmax(x)
        x = target(x)
    
        print("Example: " + str(i * batch_size) + ", test success: "
              + "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')
    
    end_testing_time = datetime.datetime.now()
    inference_time = end_testing_time - start_testing_time
    total_inference_time += inference_time
    minutes, seconds = divmod(inference_time.total_seconds(), 60)
    print("\nInference time: " + str(int(minutes)) + " min " + str(seconds) + " sec")

average_inference_time = total_inference_time / num_tests
avg_minutes, avg_seconds = divmod(average_inference_time.total_seconds(), 60)
print("\nAverage Inference time over {} iterations: {} min {} sec".format(num_tests, int(avg_minutes), avg_seconds))

# save a confusion matrix
#target.log_confusion_matrix("model_3_channels")
# Exporting weights #
#x.get_deepnet().export_network_free_parameters("./model_3_channels")

Import ./model_3_channels/Conv_3.syntxt
Import ./model_3_channels/Conv_4.syntxt
Import ./model_3_channels/Conv_5.syntxt
Import ./model_3_channels/Fc_2.syntxt
Import ./model_3_channels/Fc_3.syntxt

### Testing ###

### Testing - Iteration 1/1 ###

Example: 9984, test success: 94.50%
Inference time: 0 min 5.057639 sec

Average Inference time over 1 iterations: 0 min 5.057639 sec


## N-MNIST : Positive Events + Negative Event

In [11]:
with open('data_final/data_sum.pkl', 'rb') as file:
    data_sum = pickle.load(file)
with open('data_final/labels_sum.pkl', 'rb') as file:
    labels_sum = pickle.load(file)

In [ ]:
db = n2d2.database.Numpy(random_partitioning=False)

db.load(data_sum, labels_sum)
db.partition_stimuli(5/7, 1/7, 1/7) # training: 50k, validation: 10k, test: 10k
print("\n### Create Provider ###")
batch_size = 64
provider = n2d2.provider.DataProvider(db, [28, 28, 1], batch_size=batch_size)
print(provider)
target = n2d2.target.Score(provider)

In [ ]:
print("\n### Loading Model (without quantization) ###")
model_sum = n2d2.cells.Sequence([
    Conv(1, 6, kernel_dims=[5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(6, 16, [5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(16, 120, [4, 4], **conv_conf()),
    Fc(120, 84, **fc_conf1()),
    Fc(84, 10, **fc_conf2()),
])
print(model_sum)
softmax = n2d2.cells.Softmax(with_loss=True)

In [ ]:
nb_epochs = 15
print("\n### Training ###")

start_training_time = datetime.datetime.now()
print("Start time Training: " + str(start_training_time))
for epoch in range(nb_epochs):
    provider.set_partition("Learn")
    model_sum.learn()
    print("\n# Train Epoch: " + str(epoch) + " #")
    for i in range(math.ceil(db.get_nb_stimuli('Learn')/batch_size)):
        x = provider.read_random_batch()
        x = model_sum(x)
        x = softmax(x)
        x = target(x)
        x.back_propagate()
        x.update()
        print("Example: " + str(i * batch_size) + ", loss: "+ "{0:.3f}".format(x[0]), end='\r')
        
    print("\n### Validation ###")
    target.clear_success()
    provider.set_partition('Validation')
    model_sum.test()
    for i in range(math.ceil(db.get_nb_stimuli('Validation') / batch_size)):
        batch_idx = i*batch_size
        x = provider.read_batch(batch_idx)
        x = model_sum(x)
        x = softmax(x)
        x = target(x)
        print("Test: " + str(i * batch_size) + ", success: "+ "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')

print("\n")
end_training_time = datetime.datetime.now()
print("End time Training: " + str(end_training_time))
training_time = end_training_time - start_training_time
minutes, seconds = divmod(training_time.total_seconds(), 60)
print("Training time: " + str(int(minutes)) + " min " + str(int(seconds)) + " sec ")

In [14]:
model_sum.import_free_parameters("./model_sum", ignore_not_exists=True)
target = n2d2.target.Score(provider)
provider.set_partition('Test')
print("\n### Testing ###")

model_sum.test()

num_tests = 1
total_inference_time = datetime.timedelta()

for test_iteration in range(num_tests):
    print(f"\n### Testing - Iteration {test_iteration + 1}/{num_tests} ###\n")
    start_testing_time = datetime.datetime.now()
    for i in range(math.ceil(provider.get_database().get_nb_stimuli('Test')/batch_size)):
        batch_idx = i*batch_size
    
        x = provider.read_batch(batch_idx)
        x = model_sum(x)
        x = softmax(x)
        x = target(x)
    
        print("Example: " + str(i * batch_size) + ", test success: "
              + "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')
    
    end_testing_time = datetime.datetime.now()
    inference_time = end_testing_time - start_testing_time
    total_inference_time += inference_time
    minutes, seconds = divmod(inference_time.total_seconds(), 60)
    print("\nInference time: " + str(int(minutes)) + " min " + str(seconds) + " sec")

average_inference_time = total_inference_time / num_tests
avg_minutes, avg_seconds = divmod(average_inference_time.total_seconds(), 60)
print("\nAverage Inference time over {} iterations: {} min {} sec".format(num_tests, int(avg_minutes), avg_seconds))

# save a confusion matrix
#target.log_confusion_matrix("model_sum")
# Exporting weights #
#x.get_deepnet().export_network_free_parameters("./model_sum")

Import ./model_sum/Conv_6.syntxt
Import ./model_sum/Conv_7.syntxt
Import ./model_sum/Conv_8.syntxt
Import ./model_sum/Fc_4.syntxt
Import ./model_sum/Fc_5.syntxt

### Testing ###

### Testing - Iteration 1/1 ###

Example: 9984, test success: 93.98%
Inference time: 0 min 4.308495 sec

Average Inference time over 1 iterations: 0 min 4.308495 sec


## N-MNIST : Positive Events - Negative Events

In [15]:
with open('data_final/data_sub.pkl', 'rb') as file:
    data_sub = pickle.load(file)
with open('data_final/labels_sub.pkl', 'rb') as file:
    labels_sub = pickle.load(file)

In [ ]:
db = n2d2.database.Numpy(random_partitioning=False)

db.load(data_sub, labels_sub)
db.partition_stimuli(5/7, 1/7, 1/7) # training: 50k, validation: 10k, test: 10k

print("\n### Create Provider ###")
batch_size = 64
provider = n2d2.provider.DataProvider(db, [28, 28, 1], batch_size=batch_size)
print(provider)
target = n2d2.target.Score(provider)

In [ ]:
print("\n### Loading Model (without quantization) ###")
model_sub = n2d2.cells.Sequence([
    Conv(1, 6, kernel_dims=[5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(6, 16, [5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(16, 120, [4, 4], **conv_conf()),
    Fc(120, 84, **fc_conf1()),
    Fc(84, 10, **fc_conf2()),
])
print(model_sub)
softmax = n2d2.cells.Softmax(with_loss=True)

In [ ]:
nb_epochs = 15
print("\n### Training ###")

start_training_time = datetime.datetime.now()
print("Start time Training: " + str(start_training_time))
for epoch in range(nb_epochs):
    provider.set_partition("Learn")
    model_sub.learn()
    print("\n# Train Epoch: " + str(epoch) + " #")
    for i in range(math.ceil(db.get_nb_stimuli('Learn')/batch_size)):
        x = provider.read_random_batch()
        x = model_sub(x)
        x = softmax(x)
        x = target(x)
        x.back_propagate()
        x.update()
        print("Example: " + str(i * batch_size) + ", loss: "+ "{0:.3f}".format(x[0]), end='\r')
        
    print("\n### Validation ###")
    target.clear_success()
    provider.set_partition('Validation')
    model_sub.test()
    for i in range(math.ceil(db.get_nb_stimuli('Validation') / batch_size)):
        batch_idx = i*batch_size
        x = provider.read_batch(batch_idx)
        x = model_sub(x)
        x = softmax(x)
        x = target(x)
        print("Test: " + str(i * batch_size) + ", success: "+ "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')
        
print("\n")       
end_training_time = datetime.datetime.now()
print("End time Training: " + str(end_training_time))
training_time = end_training_time - start_training_time
minutes, seconds = divmod(training_time.total_seconds(), 60)
print("Training time: " + str(int(minutes)) + " min " + str(int(seconds)) + " sec ")

In [18]:
model_sub.import_free_parameters("./model_sub", ignore_not_exists=True)
target = n2d2.target.Score(provider)
provider.set_partition('Test')
print("\n### Testing ###")

model_sub.test()

num_tests = 1
total_inference_time = datetime.timedelta()

for test_iteration in range(num_tests):
    print(f"\n### Testing - Iteration {test_iteration + 1}/{num_tests} ###\n")
    start_testing_time = datetime.datetime.now()
    for i in range(math.ceil(provider.get_database().get_nb_stimuli('Test')/batch_size)):
        batch_idx = i*batch_size
    
        x = provider.read_batch(batch_idx)
        x = model_sub(x)
        x = softmax(x)
        x = target(x)
    
        print("Example: " + str(i * batch_size) + ", test success: "
              + "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')
    
    end_testing_time = datetime.datetime.now()
    inference_time = end_testing_time - start_testing_time
    total_inference_time += inference_time
    minutes, seconds = divmod(inference_time.total_seconds(), 60)
    print("\nInference time: " + str(int(minutes)) + " min " + str(seconds) + " sec")

average_inference_time = total_inference_time / num_tests
avg_minutes, avg_seconds = divmod(average_inference_time.total_seconds(), 60)
print("\nAverage Inference time over {} iterations: {} min {} sec".format(num_tests, int(avg_minutes), avg_seconds))

# save a confusion matrix
#target.log_confusion_matrix("model_sub")
# Exporting weights #
#x.get_deepnet().export_network_free_parameters("./model_sub")

Import ./model_sub/Conv_9.syntxt
Import ./model_sub/Conv_10.syntxt
Import ./model_sub/Conv_11.syntxt
Import ./model_sub/Fc_6.syntxt
Import ./model_sub/Fc_7.syntxt

### Testing ###

### Testing - Iteration 1/1 ###

Example: 9984, test success: 94.85%
Inference time: 0 min 4.351097 sec

Average Inference time over 1 iterations: 0 min 4.351097 sec


## N-MNIST : Only Positive Events

In [19]:
with open('data_final/data_positive.pkl', 'rb') as file:
    data_positive = pickle.load(file)
with open('data_final/labels_positive.pkl', 'rb') as file:
    labels_positive = pickle.load(file)

In [ ]:
db = n2d2.database.Numpy(random_partitioning=False)

db.load(data_positive, labels_positive)
db.partition_stimuli(5/7, 1/7, 1/7) # training: 50k, validation: 10k, test: 10k

print("\n### Create Provider ###")
batch_size = 64
provider = n2d2.provider.DataProvider(db, [28, 28, 1], batch_size=batch_size)
print(provider)
target = n2d2.target.Score(provider)

In [ ]:
print("\n### Loading Model (without quantization) ###")
model_positive = n2d2.cells.Sequence([
    Conv(1, 6, kernel_dims=[5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(6, 16, [5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(16, 120, [4, 4], **conv_conf()),
    Fc(120, 84, **fc_conf1()),
    Fc(84, 10, **fc_conf2()),
])
print(model_positive)
softmax = n2d2.cells.Softmax(with_loss=True)

In [ ]:
nb_epochs = 15
print("\n### Training ###")

start_training_time = datetime.datetime.now()
print("Start time Training: " + str(start_training_time))
for epoch in range(nb_epochs):
    provider.set_partition("Learn")
    model_positive.learn()
    print("\n# Train Epoch: " + str(epoch) + " #")
    for i in range(math.ceil(db.get_nb_stimuli('Learn')/batch_size)):
        x = provider.read_random_batch()
        x = model_positive(x)
        x = softmax(x)
        x = target(x)
        x.back_propagate()
        x.update()
        print("Example: " + str(i * batch_size) + ", loss: "+ "{0:.3f}".format(x[0]), end='\r')
        
    print("\n### Validation ###")
    target.clear_success()
    provider.set_partition('Validation')
    model_positive.test()
    for i in range(math.ceil(db.get_nb_stimuli('Validation') / batch_size)):
        batch_idx = i*batch_size
        x = provider.read_batch(batch_idx)
        x = model_positive(x)
        x = softmax(x)
        x = target(x)
        print("Test: " + str(i * batch_size) + ", success: "+ "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')
    
print("\n")        
end_training_time = datetime.datetime.now()
print("End time Training: " + str(end_training_time))
training_time = end_training_time - start_training_time
minutes, seconds = divmod(training_time.total_seconds(), 60)
print("Training time: " + str(int(minutes)) + " min " + str(int(seconds)) + " sec ")

In [22]:
model_positive.import_free_parameters("./model_positive", ignore_not_exists=True)
target = n2d2.target.Score(provider)
provider.set_partition('Test')
print("\n### Testing ###")

model_positive.test()

num_tests = 1
total_inference_time = datetime.timedelta()

for test_iteration in range(num_tests):
    print(f"\n### Testing - Iteration {test_iteration + 1}/{num_tests} ###\n")
    start_testing_time = datetime.datetime.now()
    for i in range(math.ceil(provider.get_database().get_nb_stimuli('Test')/batch_size)):
        batch_idx = i*batch_size
    
        x = provider.read_batch(batch_idx)
        x = model_positive(x)
        x = softmax(x)
        x = target(x)
    
        print("Example: " + str(i * batch_size) + ", test success: "
              + "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')
    
    end_testing_time = datetime.datetime.now()
    inference_time = end_testing_time - start_testing_time
    total_inference_time += inference_time
    minutes, seconds = divmod(inference_time.total_seconds(), 60)
    print("\nInference time: " + str(int(minutes)) + " min " + str(seconds) + " sec")

average_inference_time = total_inference_time / num_tests
avg_minutes, avg_seconds = divmod(average_inference_time.total_seconds(), 60)
print("\nAverage Inference time over {} iterations: {} min {} sec".format(num_tests, int(avg_minutes), avg_seconds))

# save a confusion matrix
#target.log_confusion_matrix("model_positive")
# Exporting weights #
#x.get_deepnet().export_network_free_parameters("./model_positive")

Import ./model_positive/Conv_12.syntxt
Import ./model_positive/Conv_13.syntxt
Import ./model_positive/Conv_14.syntxt
Import ./model_positive/Fc_8.syntxt
Import ./model_positive/Fc_9.syntxt

### Testing ###

### Testing - Iteration 1/1 ###

Example: 9984, test success: 93.21%
Inference time: 0 min 4.595262 sec

Average Inference time over 1 iterations: 0 min 4.595262 sec


## N-MNIST : Only Negative Events

In [23]:
with open('data_final/data_negative.pkl', 'rb') as file:
    data_negative = pickle.load(file)
with open('data_final/labels_negative.pkl', 'rb') as file:
    labels_negative = pickle.load(file)

In [ ]:
db = n2d2.database.Numpy(random_partitioning=False)

db.load(data_negative, labels_negative)
db.partition_stimuli(5/7, 1/7, 1/7) # training: 50k, validation: 10k, test: 10k

print("\n### Create Provider ###")
batch_size = 64
provider = n2d2.provider.DataProvider(db, [28, 28, 1], batch_size=batch_size)
print(provider)
target = n2d2.target.Score(provider)

In [ ]:
print("\n### Loading Model (without quantization) ###")
model_negative = n2d2.cells.Sequence([
    Conv(1, 6, kernel_dims=[5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(6, 16, [5, 5], **conv_conf()),
    Pool2d(pool_dims=[2, 2], stride_dims=[2, 2], pooling="Average"),
    Conv(16, 120, [4, 4], **conv_conf()),
    Fc(120, 84, **fc_conf1()),
    Fc(84, 10, **fc_conf2()),
])
print(model_negative)
softmax = n2d2.cells.Softmax(with_loss=True)

In [ ]:
nb_epochs = 15
print("\n### Training ###")

start_training_time = datetime.datetime.now()
print("Start time Training: " + str(start_training_time))
for epoch in range(nb_epochs):
    provider.set_partition("Learn")
    model_negative.learn()
    print("\n# Train Epoch: " + str(epoch) + " #")
    for i in range(math.ceil(db.get_nb_stimuli('Learn')/batch_size)):
        x = provider.read_random_batch()
        x = model_negative(x)
        x = softmax(x)
        x = target(x)
        x.back_propagate()
        x.update()
        print("Example: " + str(i * batch_size) + ", loss: "+ "{0:.3f}".format(x[0]), end='\r')
        
    print("\n### Validation ###")
    target.clear_success()
    provider.set_partition('Validation')
    model_negative.test()
    for i in range(math.ceil(db.get_nb_stimuli('Validation') / batch_size)):
        batch_idx = i*batch_size
        x = provider.read_batch(batch_idx)
        x = model_negative(x)
        x = softmax(x)
        x = target(x)
        print("Test: " + str(i * batch_size) + ", success: "+ "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')

print("\n")
end_training_time = datetime.datetime.now()
print("End time Training: " + str(end_training_time))
training_time = end_training_time - start_training_time
minutes, seconds = divmod(training_time.total_seconds(), 60)
print("Training time: " + str(int(minutes)) + " min " + str(int(seconds)) + " sec ")

In [26]:
model_negative.import_free_parameters("./model_negative", ignore_not_exists=True)
target = n2d2.target.Score(provider)
provider.set_partition('Test')
print("\n### Testing ###")

model_negative.test()

num_tests = 1
total_inference_time = datetime.timedelta()

for test_iteration in range(num_tests):
    print(f"\n### Testing - Iteration {test_iteration + 1}/{num_tests} ###\n")
    start_testing_time = datetime.datetime.now()
    for i in range(math.ceil(provider.get_database().get_nb_stimuli('Test')/batch_size)):
        batch_idx = i*batch_size
    
        x = provider.read_batch(batch_idx)
        x = model_negative(x)
        x = softmax(x)
        x = target(x)
    
        print("Example: " + str(i * batch_size) + ", test success: "
              + "{0:.2f}".format(100 * target.get_average_success()) + "%", end='\r')
    
    end_testing_time = datetime.datetime.now()
    inference_time = end_testing_time - start_testing_time
    total_inference_time += inference_time
    minutes, seconds = divmod(inference_time.total_seconds(), 60)
    print("\nInference time: " + str(int(minutes)) + " min " + str(seconds) + " sec")

average_inference_time = total_inference_time / num_tests
avg_minutes, avg_seconds = divmod(average_inference_time.total_seconds(), 60)
print("\nAverage Inference time over {} iterations: {} min {} sec".format(num_tests, int(avg_minutes), avg_seconds))

# save a confusion matrix
#target.log_confusion_matrix("model_negative")
# Exporting weights #
#x.get_deepnet().export_network_free_parameters("./model_negative")

Import ./model_negative/Conv_15.syntxt
Import ./model_negative/Conv_16.syntxt
Import ./model_negative/Conv_17.syntxt
Import ./model_negative/Fc_10.syntxt
Import ./model_negative/Fc_11.syntxt

### Testing ###

### Testing - Iteration 1/1 ###

Example: 9984, test success: 93.78%
Inference time: 0 min 5.329224 sec

Average Inference time over 1 iterations: 0 min 5.329224 sec
